In [3]:
%run "QS_1.ipynb"
transaction['DATE'] = pd.to_datetime(transaction['DATE'], origin='1899-12-30', unit='D')
transaction['year_mm']=transaction['DATE'].dt.strftime('%Y%m').astype(int)
ts=transaction.groupby(['year_mm','STORE_NBR'])['TOT_SALES'].sum().reset_index()
NoC=transaction.groupby(['year_mm','STORE_NBR'])['LYLTY_CARD_NBR'].nunique().reset_index()
TxNo=transaction.groupby(['year_mm','STORE_NBR'])['TXN_ID'].nunique().reset_index()
Pq=transaction.groupby(['year_mm','STORE_NBR'])['PROD_QTY'].sum().reset_index()
matrix=pd.merge(ts,NoC,on=['year_mm','STORE_NBR'])
matrix=pd.merge(TxNo,matrix,on=['year_mm','STORE_NBR'])
matrix=pd.merge(Pq,matrix,on=['year_mm','STORE_NBR'])
matrix.rename(columns={
    'STORE_NBR':'Store Number',
    'PROD_QTY':'Total Chips',
    'TXN_ID':' Number of Transactions',
    'LYLTY_CARD_NBR':'Number of Customers',
    'TOT_SALES':'Total Sales'},inplace=True)
from scipy.stats import ttest_ind
def uplift(trial,control):
    Trial_pre=matrix[(matrix['Store Number']==trial)&(matrix['year_mm']<201902)]
    Trial_post=matrix[(matrix['Store Number']==trial)&(matrix['year_mm']>201902)]
    control_pre=matrix[(matrix['Store Number']==control)&(matrix['year_mm']<201902)]
    control_post=matrix[(matrix['Store Number']==control)&(matrix['year_mm']>201902)]
## grouping:
    Trial_gr=Trial_pre.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
    control_gr=control_pre.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
    Trial_go=Trial_post.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
    control_go=control_post.groupby('year_mm').agg({'Total Sales':'sum','Number of Customers':'sum'}).reset_index()
# merging:
    pre=pd.merge(Trial_gr,control_gr,on='year_mm')
    post=pd.merge(Trial_go,control_go,on='year_mm')
#calculating the ratios:
    pre['ratio_ts']=pre['Total Sales_x']/pre['Total Sales_y']
    pre['ratio_Noc']=pre['Number of Customers_x']/pre['Number of Customers_y']
    post['ratio_ts']=post['Total Sales_x']/post['Total Sales_y']
    post['ratio_Noc']=post['Number of Customers_x']/post['Number of Customers_y']
#Calculating test statistic and p value:
    t_stat_cust, p_value_cust = ttest_ind(post['ratio_Noc'], pre['ratio_Noc'], equal_var=False)
    return{
    'Trial Store':trial,
    'Control Store':control,
    'T- test statistic': round(t_stat_cust, 2),
    'P-value': round(p_value_cust, 2),
    'Significant': 'Yes' if p_value_cust < 0.05 else 'No'
    }     
trial = [77, 86, 88]
control = [233, 155, 237]
results_customer = []
for trial, control in zip(trial, control):
    results_customer.append(uplift(trial, control))

# Converting the outcome to DataFrame
test_summary_customer = pd.DataFrame(results_customer)
print (test_summary_customer)


   Trial Store  Control Store  T- test statistic  P-value Significant
0           77            233               1.36     0.27          No
1           86            155               1.40     0.25          No
2           88            237               1.85     0.16          No
